**LVV-T1602: Integration of Camera Rotator with SAL**

This notebook runs the test cases in [LVV-1602](https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1602).
The objective of this test case is to verify the software requirements of the camera rotator, as defined in LTS-160.
The test will be performed on summit with the real hardware.
Before that, the Tucson test stand is used to develop the test procedure.
The rotator commandable SAL component (CSC) is used to control the rotator hardware.
The published event and telemetry are stored in the engineering facility database (EFD).

It is noted that the **MTMount** telemetry data is required.
Otherwise, the rotator CSC will put the low-level controller into the **Fault** state.

The **setup** notebook can be used to bring up rotator and mount the camera cable wrapper (CCW).

**Import Modules**

In [ ]:
%matplotlib inline
%matplotlib widget
import asyncio
from datetime import datetime
import time
from astropy.time import Time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lsst.ts import salobj
from lsst.ts import simactuators

**Custom Functions**

In [ ]:
def script_log_start(controller_script, test_title, test_detail):
    """Log the start of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """

    time_now = datetime.now()
    controller_script.log.info(f"START -- {test_title} -- LVV-T1602 -- {test_detail} -- Starting Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [ ]:
def script_log_stop(controller_script, test_title, test_detail):
    """Log the stop of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """

    time_now = datetime.now()
    controller_script.log.info(f"STOP -- {test_title} -- LVV-T1602 -- {test_detail} -- Stopping Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [ ]:
async def track_target_time(csc, start_position, velocity, lasting_time):
    """Track the target with a lasting time.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    start_position : float
        Starting position of ramp (deg).
    velocity : float
        Velocity of motion along the ramp (deg/sec).
    lasting_time : float or int
        Lasting time of tracking target. This value should be >= 0.
    """

    end_position = start_position + velocity * lasting_time
    await track_target(csc, start_position, end_position, velocity)

In [ ]:
async def track_target(csc, start_position, end_position, velocity):
    """Track the target.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    start_position : float
        Starting position of ramp (deg).
    end_position : float
        Ending position of ramp (deg).
    velocity : float
        Velocity of motion along the ramp (deg/sec).
    """    

    await make_sure_csc_enabled(csc)
    
    # 20 Hz is 0.05 ms
    track_advance_time = 0.05
    ramp_generator = simactuators.RampGenerator(
        [start_position], [end_position], [velocity], track_advance_time)

    await csc.cmd_trackStart.set_start(timeout=15)
    await asyncio.sleep(0.15)
    for positions, velocities, tai in ramp_generator():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=10,
        )
        await asyncio.sleep(track_advance_time)

    await asyncio.sleep(1)

In [ ]:
async def move_origin(csc):
    """Move to the origin.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """

    sleep_time = 3
    await asyncio.sleep(sleep_time)

    await make_sure_csc_enabled(csc)

    # Workaround the problem of MOOG state machine to decide a new point-to-point movement
    await csc.cmd_move.set_start(position=0.1, timeout=90)
    await asyncio.sleep(sleep_time)

    await make_sure_csc_enabled(csc)

    await csc.cmd_move.set_start(position=0, timeout=90)
    await asyncio.sleep(sleep_time)

In [ ]:
async def make_sure_csc_enabled(csc):
    """Make sure the CSC is enabled.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """

    await asyncio.sleep(3)
    data = csc.evt_summaryState.get()
    await asyncio.sleep(3)
    if data.summaryState != salobj.State.ENABLED:
        await salobj.set_summary_state(csc, salobj.State.ENABLED, timeout=90)

In [ ]:
async def print_summary_state(csc):
    """
    Print the summary state of rotator.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """

    data = csc.evt_summaryState.get()
    print(salobj.State(data.summaryState))

**Instantiate the EFD and CSC**

Need to make sure the **ospl** is running for the data distribution system (DDS) communication.

In [ ]:
domain = salobj.Domain()
rotator = salobj.Remote(domain, "MTRotator")
await rotator.start_task
await rotator.cmd_setLogLevel.set_start(level=10)

**MTMount** controller is used to run CCW.
Put it into the **Enabled** state and follow the rotator.

In [ ]:
mtmount = salobj.Remote(domain, "MTMount")
await mtmount.start_task

await make_sure_csc_enabled(mtmount)

await mtmount.cmd_enableCameraCableWrapFollowing.set_start()

**Setup the Script**

In [ ]:
time_start_testcases = datetime.now()
test_title = "Run the LVV-T1602"
script = salobj.Controller("Script", index=10)

# Wait 10 second may help with DDS problems; closing all other kernels may help too
await asyncio.sleep(10)
print(f"Time to start the test cases is {time_start_testcases}.")

**Step 1**

Camera Rotator DDS Startup Procedure. Exercise the LVV-T1597 to use the EUI.

**Step 2**

Verify the TCP/IP is connected to the low-level controller.

In [ ]:
time_start_step2 = script_log_start(script, test_title, "Step 2")

Verify that the **MTRotator_logevent_connected** event is True.

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.DISABLED, timeout=10)
await asyncio.sleep(2)

In [ ]:
data = rotator.evt_connected.get()
print(data.connected)

Verify that the **MTRotator_logevent_connected** event is False.

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.STANDBY, timeout=10)
await asyncio.sleep(2)

In [ ]:
data = rotator.evt_connected.get()
print(data.connected)

In [ ]:
time_stop_step2 = script_log_stop(script, test_title, "Step 2")

**Step 3**

Verify the **MTRotator_logevent_configuration** event.

In [ ]:
time_start_step3 = script_log_start(script, test_title, "Step 3")

Verify the **MTRotator_logevent_configuration** event publishes the data.

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.DISABLED, timeout=10)
await asyncio.sleep(2)

In [ ]:
data = rotator.evt_configuration.get()
print(data)

Need to be in the **ENABLED** state to update the configuration.

In [ ]:
await make_sure_csc_enabled(rotator)
await rotator.cmd_configureVelocity.set_start(vlimit=3.0, timeout=10)
await asyncio.sleep(2)

Verify the **MTRotator_logevent_configuration** event publishes the changed data.

In [ ]:
data = rotator.evt_configuration.get()
print(data)

Return to the previous configuration.

In [ ]:
await rotator.cmd_configureVelocity.set_start(vlimit=3.5, timeout=10)
await asyncio.sleep(2)

await salobj.set_summary_state(rotator, salobj.State.DISABLED, timeout=10)

Log to the PXI computer and modify the configuration (maximum acceleration).
Need to put the rotator CSC into the **ENABLED** state to read the configuration file.

In [ ]:
await make_sure_csc_enabled(rotator)

Verify the **MTRotator_logevent_configuration** event publishes the changed data.

In [ ]:
data = rotator.evt_configuration.get()
print(data)

Put the rotator to the **DISABLED** state.
Log in the PXI controller to modify the configuration back.

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.DISABLED, timeout=10)

In [ ]:
time_stop_step3 = script_log_stop(script, test_title, "Step 3")

**Step 4**

Verify the **MTRotator_logevent_interlock** event.

In [ ]:
time_start_step4 = script_log_start(script, test_title, "Step 4")

Press the E-Stop.

Verify the **MTRotator_logevent_interlock** event is True.

In [ ]:
await asyncio.sleep(2)

data = rotator.evt_interlock.get()
print(data.engaged)

Release the E-Stop (contains the CCW EUI part).

Verifiy that the CSC is in the **FAULT** state

In [ ]:
await print_summary_state(rotator)

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.STANDBY, timeout=10)

Verify the **MTRotator_logevent_interlock** event is False.

In [ ]:
await asyncio.sleep(2)

data = rotator.evt_interlock.get()
print(data.engaged)

In [ ]:
time_stop_step4 = script_log_stop(script, test_title, "Step 4")

**Step 5**

GUI should be able to take over the control from CSC.

In [ ]:
time_start_step5 = script_log_start(script, test_title, "Step 5")

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.DISABLED, timeout=10)

Verify the **MTRotator_logevent_commandableByDDS** event is True.

In [ ]:
await asyncio.sleep(2)

data = rotator.evt_commandableByDDS.get()
print(data.state)

Use the EUI to do the state transition.

Verify the **MTRotator_logevent_commandableByDDS** event is False.

In [ ]:
await asyncio.sleep(2)

data = rotator.evt_commandableByDDS.get()
print(data.state)

In [ ]:
time_stop_step5 = script_log_stop(script, test_title, "Step 5")

**Step 6**

Back To DDS Mode.

In [ ]:
time_start_step6 = script_log_start(script, test_title, "Step 6")

Use the EUI to change the commander to be DDS.
Make sure the controller state is **Offline/Available** state.

Verify the **MTRotator_logevent_commandableByDDS** event is True.

In [ ]:
await asyncio.sleep(2)

data = rotator.evt_commandableByDDS.get()
print(data.state)

In [ ]:
time_stop_step6 = script_log_stop(script, test_title, "Step 6")

**Step 7**

Rotator point-to-point movement.

In [ ]:
time_start_step7 = script_log_start(script, test_title, "Step 7")

Verify the **MTRotator_ccwFollowingError** telemetry by Chronograph.

In [ ]:
await make_sure_csc_enabled(rotator)

In [ ]:
for position in (5, 0):
    await rotator.cmd_move.set_start(position=position, timeout=90)
    await asyncio.sleep(10)

Verify that **positionError** and **velocityError** in **MTRotator_ccwFollowingError** telemetry different from 0 are recorded by Chronograph.

In [ ]:
time_stop_step7 = script_log_stop(script, test_title, "Step 7")

**Step 8**

Verify the **MTRotator_rotation** telemetry data.

Check the rotator data performed at step 7.

**Step 9**

Verify the **MTRotator_electrical** telemetry data.

Check the rotator data performed at step 7.

**Step 10**

Verify the **MTRotator_motors** telemetry.

Check the rotator data performed at step 7.

**Step 11**

Run the **Camera Rotator Software Re-verification.ipynb**.

Use a Jupyter notebook to execute the tests in this test case:
Use the latest version available on GitHub for: **Camera Rotator Software Re-verification.ipynb**.

**Step 12**

Test sequence 1 – **move** command.

Record the current position of the rotator by EUI or Chronograph.

**Step 13**

In [ ]:
time_start_step13 = script_log_start(script, test_title, "Step 13")

In [ ]:
await make_sure_csc_enabled(rotator)

positions = [1, 13, 50, -1, -13, -50]
sleep_times = [1, 10, 30, 1, 10, 30]
for position, sleep_time in zip(positions, sleep_times):
    # Move to position
    await rotator.cmd_move.set_start(position=position, timeout=90)
    await asyncio.sleep(sleep_time)

    # Back to origin
    await rotator.cmd_move.set_start(position=0, timeout=90)
    await asyncio.sleep(sleep_time)

In [ ]:
time_stop_step13 = script_log_stop(script, test_title, "Step 13")

**Step 14**

Verify the **MTRotator_logevent_controllerState** event were generated.

Check the rotator data performed at step 13.

**Step 15** and **Step 16**

Test sequence 2 - **stop** command.

In [ ]:
time_start_step15 = script_log_start(script, test_title, "Step 15 and 16")

In [ ]:
await rotator.cmd_move.set_start(position=50, timeout=90)
await asyncio.sleep(3)

await rotator.cmd_stop.set_start(timeout=90)
await asyncio.sleep(5)

In [ ]:
time_stop_step15 = script_log_stop(script, test_title, "Step 15 and 16")

**Step 17**

In [ ]:
time_start_step17 = script_log_start(script, test_title, "Step 17")

Record the current position of the rotator by EUI or Chronograph.

In [ ]:
await rotator.cmd_move.set_start(position=-60, timeout=90)
await asyncio.sleep(30)

Move the rotator back to the origin.

In [ ]:
await rotator.cmd_move.set_start(position=0, timeout=90)
await asyncio.sleep(30)

In [ ]:
time_stop_step17 = script_log_stop(script, test_title, "Step 17")

**Step 18**

Record the corresponding DDS events that were generated.

**Step 19**

Test **trackStart** and **track** command

In [ ]:
time_start_step19 = script_log_start(script, test_title, "Step 19")

In [ ]:
await rotator.cmd_trackStart.set_start(timeout=15)
await asyncio.sleep(3)

await make_sure_csc_enabled(rotator)

In [ ]:
time_stop_step19 = script_log_stop(script, test_title, "Step 19")

**Step 20**

Test sequence 4 - **track** and **trackStart** commands - positive direction

In [ ]:
time_start_step20 = script_log_start(script, test_title, "Step 20")

In [ ]:
await move_origin(rotator)
await track_target_time(rotator, 10, 0.068, 20)

await asyncio.sleep(3)
await make_sure_csc_enabled(rotator)

In [ ]:
time_stop_step20 = script_log_stop(script, test_title, "Step 20")

**Step 21**

Record the **MTRotator_logevent_inPosition** and **MTRotator_logevent_controllerState** events that were generated.
This is easier to compare with the movement after the test.

**Step 22**

Test sequence 4 - **track** and **trackStart** commands - negative direction

In [ ]:
time_start_step22 = script_log_start(script, test_title, "Step 22")

In [ ]:
await move_origin(rotator)
await track_target_time(rotator, -10, -0.068, 20)

await asyncio.sleep(3)
await make_sure_csc_enabled(rotator)

In [ ]:
time_stop_step22 = script_log_stop(script, test_title, "Step 22")

**Step 23**

Record the **MTRotator_logevent_inPosition** and **MTRotator_logevent_controllerState** events that were generated.
This is easier to compare with the movement after the test.

**Step 24**

Test sequence 6 - **configureVelocity** command

In [ ]:
time_start_step24 = script_log_start(script, test_title, "Step 24")

In [ ]:
await rotator.cmd_configureVelocity.set_start(vlimit=4.0, timeout=10)
await asyncio.sleep(2)

In [ ]:
time_stop_step24 = script_log_stop(script, test_title, "Step 24")

**Step 25**

In [ ]:
time_start_step25 = script_log_start(script, test_title, "Step 25")

In [ ]:
await rotator.cmd_configureVelocity.set_start(vlimit=0.5, timeout=10)
await asyncio.sleep(2)

In [ ]:
time_stop_step25 = script_log_stop(script, test_title, "Step 25")

**Step 26**

In [ ]:
time_start_step26 = script_log_start(script, test_title, "Step 26")

In [ ]:
await move_origin(rotator)

await rotator.cmd_move.set_start(position=10, timeout=90)
await asyncio.sleep(35)

In [ ]:
# Move back to origin (this will be long)

await rotator.cmd_move.set_start(position=0, timeout=90)
await asyncio.sleep(35)

In [ ]:
time_stop_step26 = script_log_stop(script, test_title, "Step 26")

**Step 27**

Record the **MTRotator_logevent_inPosition**.
This is easier to compare with the movement after the test.

**Step 28**

Test sequence 7 - **configureAcceleration** command

In [ ]:
time_start_step28 = script_log_start(script, test_title, "Step 28")

In [ ]:
await rotator.cmd_configureAcceleration.set_start(alimit=2.0, timeout=10)
await asyncio.sleep(2)

In [ ]:
time_stop_step28 = script_log_stop(script, test_title, "Step 28")

**Step 29**

In [ ]:
time_start_step29 = script_log_start(script, test_title, "Step 29")

In [ ]:
await rotator.cmd_configureAcceleration.set_start(alimit=0.5, timeout=10)
await asyncio.sleep(2)

In [ ]:
time_stop_step29 = script_log_stop(script, test_title, "Step 29")

**Step 30**

In [ ]:
time_start_step30 = script_log_start(script, test_title, "Step 30")

In [ ]:
await rotator.cmd_move.set_start(position=10, timeout=90)
await asyncio.sleep(50)

In [ ]:
time_stop_step30 = script_log_stop(script, test_title, "Step 30")

Recover the configuration and system.

In [ ]:
await rotator.cmd_configureAcceleration.set_start(alimit=1.0, timeout=10)
await asyncio.sleep(2)

await rotator.cmd_configureVelocity.set_start(vlimit=3.5, timeout=10)
await asyncio.sleep(2)

await move_origin(rotator)

**Step 31**

Record the **MTRotator_logevent_inPosition**.
This is easier to compare with the movement after the test.

**Step 32**

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.STANDBY, timeout=10)

Use the EUI to transition the rotator controller into **Offline/PublishOnly** state.

**Step 33**

Rotator action on state commands

Transition the low-level controller state machine to **Offline/Available** state using the EUI.

**Step 34**

From the EUI, select the DDS button to allow the system to receive commands from DDS.

**Step 35**

In [ ]:
time_start_step35 = script_log_start(script, test_title, "Step 35")

In [ ]:
await rotator.cmd_start.set_start(timeout=10)

In [ ]:
time_stop_step35 = script_log_stop(script, test_title, "Step 35")

**Step 36**

In [ ]:
time_start_step36 = script_log_start(script, test_title, "Step 36")

In [ ]:
await rotator.cmd_enable.set_start(timeout=10)

In [ ]:
time_stop_step36 = script_log_stop(script, test_title, "Step 36")

**Step 37**

In [ ]:
time_start_step37 = script_log_start(script, test_title, "Step 37")

In [ ]:
await rotator.cmd_disable.set_start(timeout=10)

In [ ]:
time_stop_step37 = script_log_stop(script, test_title, "Step 37")

**Step 38**

In [ ]:
time_start_step38 = script_log_start(script, test_title, "Step 38")

In [ ]:
await rotator.cmd_standby.set_start(timeout=10)

In [ ]:
time_stop_step38 = script_log_stop(script, test_title, "Step 38")

**Step 39**

Unplug the encoder cable.

In [ ]:
time_start_step39 = script_log_start(script, test_title, "Step 39")

In [ ]:
await make_sure_csc_enabled(rotator)

Unplug an encoder cable for one of the rotator motors.

In [ ]:
time_stop_step39 = script_log_stop(script, test_title, "Step 39")

**Step 40**

In [ ]:
time_start_step40 = script_log_start(script, test_title, "Step 40")

Re-plug the encoder cable.

In [ ]:
await rotator.cmd_standby.set_start(timeout=10)

In [ ]:
time_stop_step40 = script_log_stop(script, test_title, "Step 40")

**Step 41**

Unplug the linear encoder cable.

In [ ]:
time_start_step41 = script_log_start(script, test_title, "Step 41")

In [ ]:
await make_sure_csc_enabled(rotator)

Unplug a linear encoder cable for the rotator.

Replug a linear encoder cable for the rotator.

In [ ]:
await rotator.cmd_standby.set_start(timeout=10)

In [ ]:
time_stop_step41 = script_log_stop(script, test_title, "Step 41")

**Step 42**

In [ ]:
time_start_step42 = script_log_start(script, test_title, "Step 42")

Set the **Following Error Threshold** parameter to a very small value (0.0001 deg or smaller).
Adjust this value within the low-controller configuration file.

In [ ]:
await make_sure_csc_enabled(rotator)

In [ ]:
time_stop_step42 = script_log_stop(script, test_title, "Step 42")

**Step 43**

In [ ]:
time_start_step43 = script_log_start(script, test_title, "Step 43")

In [ ]:
await rotator.cmd_move.set_start(position=10, timeout=90)
await asyncio.sleep(5)

In [ ]:
time_stop_step43 = script_log_stop(script, test_title, "Step 43")

Recover the controller settings.

Set the **Following Error Threshold** parameter back to the original value.

In [ ]:
await rotator.cmd_standby.set_start(timeout=10)

**Step 44**

Positive software limit test

In [ ]:
time_start_step44 = script_log_start(script, test_title, "Step 44")

Set the positive and negative software limits to 1 and -1 degree.
Adjust this value within the low-controller configuration file.

In [ ]:
await make_sure_csc_enabled(rotator)

In [ ]:
await track_target(rotator, 0.8, 1.2, 0.068)

In [ ]:
time_stop_step44 = script_log_stop(script, test_title, "Step 44")

Recover back to origin.

In [ ]:
await make_sure_csc_enabled(rotator)
await move_origin(rotator)

**Step 45**

Negative software limit test

In [ ]:
time_start_step45 = script_log_start(script, test_title, "Step 45")

In [ ]:
await track_target(rotator, -0.8, -1.2, -0.068)

In [ ]:
time_stop_step45 = script_log_stop(script, test_title, "Step 45")

Recover back to origin.

In [ ]:
await make_sure_csc_enabled(rotator)
await move_origin(rotator)

Return to previous configuration.

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.STANDBY, timeout=10)

Set the positive and negative software limits back to 90 and -90 degree.
Adjust this value within the low-controller configuration file.

**Step 46**

Unplug the EtherCAT cable.

In [ ]:
time_start_step46 = script_log_start(script, test_title, "Step 46")

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.DISABLED, timeout=10)

Unplug the EtherCAT cable.

Re-plug the EtherCAT cable.

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.STANDBY, timeout=10)

In [ ]:
time_stop_step46 = script_log_stop(script, test_title, "Step 46")

**Step 47**

Transition the rotator into **STANDBY** state.

In [ ]:
await salobj.set_summary_state(rotator, salobj.State.STANDBY, timeout=10)